# Part 3: Serving an ML Model

This part of the MLRun getting-started tutorial walks you through the steps for creating, deploying, and testing a model-serving function ("a serving function" a.k.a. "a model server") using MLRun and Nuclio.

MlRun serving can take MLRun models or standard model files and produce managed real-time serverless functions based on the Nuclio real-time serverless engine, which can be deployed everywhere
[Nuclio](https://nuclio.io/) is a high-performance open-source "serverless" framework that's focused on data, I/O, and compute-intensive workloads.
For more information, see the [Nuclio GitHub repository](https://github.com/nuclio/nuclio).

Simple model serving classes can be written in Python or be taken from a set of pre-developed ML/DL classes.
The code can handle complex data, feature preparation, and binary data (such as images and video files).
The Nuclio serving engine supports the full model-serving life cycle, including auto generation of microservices, APIs, load balancing, model logging and monitoring, and configuration management.

The tutorial consists of the following steps:

1. [Setup](#gs-tutorial-3-step-setup) &mdash; load your project
2. [Writing A Simple Serving Class](#gs-tutorial-3-step-writing-a-serving-class)
3. [Deploying the Model-Serving Function (Service)](#gs-tutorial-3-step-deploy-the-serving-function)
4. [Using the Model-Serving Function](#gs-tutorial-3-step-using-the-serving-function)
5. [Viewing the Nuclio Serving Function on the Dashboard](#gs-tutorial-3-step-view-serving-func-in-ui)

<a id="gs-tutorial-3-prerequisites"></a>

## Prerequisites

The following steps are a continuation of the previous parts of this getting-started tutorial and rely on the generated outputs.
Therefore, make sure to first run parts 1&mdash;[2](02-model-training.ipynb) of the tutorial.

<a id="gs-tutorial-3-step-setup"></a>

## Step 1: Setup and Configuration

<a id="gs-tutorial-3-import-libaries"></a>

### Importing Libraries

In [1]:
from os import path
import mlrun

> 2021-01-24 19:11:18,203 [warning] Failed resolving version info. Ignoring and using defaults
> 2021-01-24 19:11:19,459 [warning] Unable to parse server or client version. Assuming compatible: {'server_version': '0.6.0-rc10', 'client_version': 'unstable'}


### Initializing Your MLRun Environment

Use the `set_environment` MLRun method to configure the working environment and default configuration. 
Define a project name and set the `project` parameter, setting the `user_project` flag will add the current user to the project name (avoiding a case of multiple users using the same project name).

In [2]:
# Set the project name
project_name_base = 'getting-started-tutorial'
# Initialize your MLRun environment and save the artifacts path
project_name, artifact_path = mlrun.set_environment(project=project_name_base, user_project=True)

<a id="gs-tutorial-3-step-writing-a-serving-class"></a>

## Step 2: Writing A Simple Serving Class

The serving class is initialized automatically by the model server.
All you need is to implement two mandatory methods:

- `load` &mdash; downloads the model files and loads the model into memory.
    This can be done either synchronously or asynchronously.
- `predict` &mdash; accepts a request payload and returns prediction (inference) results.

For more detailed information on serving classes, see the [MLRun documentation](https://github.com/mlrun/mlrun/blob/release/v0.6.x-latest/mlrun/serving/README.md).

The following code demonstrates a minimal scikit-learn (a.k.a. sklearn) serving-class implementation:

In [3]:
# nuclio: start-code

In [4]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [5]:
# nuclio: end-code

<a id="gs-tutorial-3-step-deploy-the-serving-function"></a>

## Step 3: Deploying the Model-Serving Function (Service)

To provision (deploy) a function for serving the model ("a serving function") you need to create an MLRun function of type `serving`.
You can do this by using the `code_to_function` MLRun method from a web notebook, or by importing an existing serving function or template from the MLRun functions marketplace.

<a id="gs-tutorial-3-convert-serving-class-to-function"></a>

### Converting a Serving Class to a Serving Function

The following code converts the `ClassifierModel` class that you defined in the previous step to a serving function.
The name of the class to be used by the serving function is set in `spec.default_class`.

In [6]:
from mlrun import code_to_function
serving_fn = code_to_function('serving', kind='serving',image='mlrun/mlrun')
serving_fn.spec.default_class = 'ClassifierModel'

Add the model created in previous notebook by the training function  


In [7]:
model_file = f'store://{project_name}/train-iris-train_iris_model'
serving_fn.add_model('my_model',model_path=model_file)

In [8]:
serving_fn = serving_fn.apply(mlrun.mount_v3io(remote='projects',mount_path='/v3io/projects'))

<a id="gs-tutorial-3-building-and-deploying-the-serving-function"></a>

### Building and Deploying the Serving Function

Use the `deploy` method of the MLRun serving function to build and deploy a Nuclio serving function from your serving-function code.

In [9]:
serving_fn.deploy()

> 2021-01-24 19:11:22,369 [info] Starting remote function deploy
2021-01-24 19:11:22  (info) Deploying function
2021-01-24 19:11:22  (info) Building
2021-01-24 19:11:22  (info) Staging files and preparing base images
2021-01-24 19:11:22  (info) Building processor image
2021-01-24 19:11:23  (info) Build complete
2021-01-24 19:11:29  (info) Function deploy complete
> 2021-01-24 19:11:30,075 [info] function deployed, address=default-tenant.app.aefccdjffbit.iguazio-cd0.com:31537


'http://default-tenant.app.aefccdjffbit.iguazio-cd0.com:31537'

In [10]:
serving_fn.spec.base_spec.get('metadata').get('name')

'serving'

<a id="gs-tutorial-3-step-using-the-serving-function"></a>

## Step 4: Using the Model-Serving Function

After the function is deployed successfully, the serving function has a new HTTP endpoint for handling serving requests.
The example tutorial serving function receives HTTP prediction (inference) requests on this endpoint; calls the `predict` method to get the requested predictions; and returns the results on the same endpoint.

In [11]:
function_address = serving_fn.status.address
print (f'The address for the function is {function_address} \n')

!curl $function_address

The address for the function is default-tenant.app.aefccdjffbit.iguazio-cd0.com:31537 

{"name": "ModelRouter", "version": "v2", "extensions": []}

<a id="gs-tutorial-3-testing-the-model-server"></a>

### Testing the Model Server

Test your model server by sending data for inference.
The `invoke` serving-function method enables programmatic testing of the serving function.
For model inference (predictions), specify the model name followed by `infer`:
```
/v2/models/{model_name}/infer
```
For complete model-service API commands &mdash; such as for list models (`models`), get model health (`ready`), and model explanation (`explain`) &mdash; see the [MLRun documentation](https://github.com/mlrun/mlrun/blob/release/v0.6.x-latest/mlrun/serving/README.md#model-server-api).

In [12]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
serving_fn.invoke('/v2/models/my_model/infer', my_data)

{'id': '5c811695-d612-432d-92c1-f01204935971',
 'model_name': 'my_model',
 'outputs': [0, 2]}

<a id="gs-tutorial-3-step-view-serving-func-in-ui"></a>

## Step 5: Viewing the Nuclio Serving Function on the Dashboard

On the **Projects** dashboad page, select the project and then select "Real-time functions (Nuclio)".

<img src="../_static/images/nuclio-deploy.png" alt="Jobs" width="800"/>

<a id="gs-tutorial-3-step-save-project"></a>

## Done!

Congratulation! You've completed Part 3 of the MLRun getting-started tutorial.
Proceed to [Part 4](04-pipeline.ipynb) to learn how to create an automated pipeline for your project.